In [1]:
import sys
sys.path.append("/home/srelins/docs/cy_fdm_builder/src")
from testing_helpers import *
from FDM_builder import *

## TO DO:

* PRIORITY: TEST observation_period table!!
- in particular, work our what happens if you re-run .build() on a dataset


* split table and dataset objects into two sparate source files
* start process with dataset - add guidance messages for dataset creation
i.e. dataset empty, build FDMTables -- or -- dataset contain XXXXX etc...
* run dataset build over whole dataset by default - add options to specify table_ids/FDMTable objects
* will need to add "verbose" functionality to FDMTables or the console print statements will be fucking epic
* add .drop() method to table


## Testing:

In [2]:
build_test_environment()

/home/srelins/anaconda3/envs/FDMBuilder/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


LOADING: table 1
LOADING: table 2
LOADING: table 3
LOADING: table 4
LOADING: table 5
LOADING: table 6


In [3]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_1 #####
________________________________________________________________________________

* Table src_table_1 copied to CY_TESTS_FDM!


In [4]:
table_1.build(event_start_date_cols="date")

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_1 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct

2. Adding person_id column:

	* src_table_1 already contains person_id column

3. Adding event_start_date:

	* event_start_date column added!

4. Adding event_end_date:

	No event_end_date column found and event_end_date_column not specified.
	event_end_date end date will be omitted. If you wish to add an event_end_date,
	re-run .build() specifying the `event_end_date` argument.

	Note: Table build process will complete, but the observation periods derived
	from a source table with only an event start date may not be accurate, and 
	may result in erroneous observations going unidentified. Talk to the CYP data
	team if at all uncertain about ommiting the event end date.

________________________________________________________________________________

	 ##### 

In [5]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)
table_3.build()

	 ##### Initialising FDM Table - src_table_3 #####
________________________________________________________________________________

* Table src_table_3 copied to CY_TESTS_FDM!
	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_3 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* EDRN found - syntax correct

2. Adding person_id column:

	* person_id column added!

3. Adding event_start_date:

	No event_start_date column found and event_start_date_cols not specified.

	To complete the build process, an event start date is required. Specify the
	`event_start_date_cols` argument of the .build() method as either:

	* If one column contains the full event date:

	  run .build(event_start_date_cols="insert-column-name-here")
	  NOTE: The process assumes a default date format of year-month-day. If the
	  date column is formatted as day-month-year set the `event_start_date_format`
	  argument as "DMY" 

In [6]:
table_3.get_column_names()

['person_id', 'EDRN', 'month', 'year']

In [7]:
table_3.build(event_start_date_cols=["year", "month", "15"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_3 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_3 already contains person_id column

	NOTE: EDRN also found in src_table_3. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_3 and run .build() again

3. Adding event_start_date:

	* event_start_date column added!

4. Adding event_end_date:

	No event_end_date column found and event_end_date_column not specified.
	event_end_date end date will be omitted. If you wish to add an event_end_date,
	re-run .build() specifying the `event_end_date` argument.

	Note: Table build process will complete, but the observation periods derived
	from a source table with only an event start date may not be accurate, and 
	may result in erroneous obser

In [8]:
table_5 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_5",
    dataset_id = "CY_TESTS_FDM"
)
table_5.build()

	 ##### Initialising FDM Table - src_table_5 #####
________________________________________________________________________________

* Table src_table_5 copied to CY_TESTS_FDM!
	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_5 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_5 already contains person_id column

	NOTE: EDRN also found in src_table_5. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_5 and run .build() again

3. Adding event_start_date:

	No event_start_date column found and event_start_date_cols not specified.

	To complete the build process, an event start date is required. Specify the
	`event_start_date_cols` argument of the .build() method as either:

	* If one column contains the full event date:

	  run .build

In [9]:
table_5.head()

,person_id,EDRN,date_string
0,41,JQ,May-1987
1,junk_person_id_237,junk_EDRN_237,May-2000
2,45,JU,July-2000
3,54,KT,July-2002
4,58,KX,July-2020
5,51,KQ,June-2003
6,junk_person_id_109,junk_EDRN_109,April-2009
7,44,JT,March-1981
8,43,JS,March-1984
9,53,KS,March-2006


In [10]:
table_5.add_column('SPLIT(date_string, "-")[OFFSET(0)] AS Month')
table_5.head()

,person_id,EDRN,date_string,Month
0,41,JQ,May-1987,May
1,junk_person_id_237,junk_EDRN_237,May-2000,May
2,45,JU,July-2000,July
3,54,KT,July-2002,July
4,58,KX,July-2020,July
5,51,KQ,June-2003,June
6,junk_person_id_109,junk_EDRN_109,April-2009,April
7,44,JT,March-1981,March
8,43,JS,March-1984,March
9,53,KS,March-2006,March


In [11]:
table_5.add_column('SPLIT(date_string, "-")[OFFSET(1)] AS Year')
table_5.head()

,person_id,EDRN,date_string,Month,Year
0,41,JQ,May-1987,May,1987
1,junk_person_id_237,junk_EDRN_237,May-2000,May,2000
2,45,JU,July-2000,July,2000
3,54,KT,July-2002,July,2002
4,58,KX,July-2020,July,2020
5,51,KQ,June-2003,June,2003
6,junk_person_id_109,junk_EDRN_109,April-2009,April,2009
7,44,JT,March-1981,March,1981
8,43,JS,March-1984,March,1984
9,53,KS,March-2006,March,2006


In [12]:
table_5.drop_column("date_string")

In [13]:
table_5.head()

,person_id,EDRN,Month,Year
0,41,JQ,May,1987
1,junk_person_id_237,junk_EDRN_237,May,2000
2,45,JU,July,2000
3,54,KT,July,2002
4,58,KX,July,2020
5,51,KQ,June,2003
6,junk_person_id_109,junk_EDRN_109,April,2009
7,44,JT,March,1981
8,43,JS,March,1984
9,53,KS,March,2006


In [14]:
table_5.build(event_start_date_cols = ["15", "Month", "Year"], event_start_date_format="DMY", 
              event_end_date_cols = ["15", "Month", "Year"], event_end_date_format="DMY")

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_5 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_5 already contains person_id column

	NOTE: EDRN also found in src_table_5. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_5 and run .build() again

3. Adding event_start_date:

	* event_start_date column added!

4. Adding event_end_date:

	* event_end_date column added!

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_5 COMPLETE! #####



In [15]:
dataset = FDMDataset(dataset_id="CY_TESTS_FDM", fdm_tables=[table_1, table_3, table_5])

In [16]:
dataset.build()

		 ##### BUILDING FDM DATASET CY_TESTS_FDM #####
________________________________________________________________________________

1. Checking source input tables:

	* src_table_1 - OK
	* src_table_3 - OK
	* src_table_5 - OK

2. Building person table:

	* Joining data from master person table
	* Person table built!

3. Building table of individuals with no person_id

	* individuals_missing_person_id created with 5 entries

4. Building person_ids missing from master table

	* person_ids_missing_from_master created with 10 entries

5. Building initial observation_period table

	* observation_period table built

6. Removing entries outside observation period

	* entries outside observation period removed from src_table_1
	  and stored in src_table_1_outside_obs
	* entries outside observation period removed from src_table_3
	  and stored in src_table_3_outside_obs
	* entries outside observation period removed from src_table_5
	  and stored in src_table_5_outside_obs

7. Rebuilding person t

In [17]:
blah = False

print(f"""
    hello {"and goodbye" 
           if blah else ""} - boop
""")


    hello  - boop

